### Aula 18 - Calculando o ângulo móvel de uma série
#### Leandro Guerra - Outspoken Market
Quer publicar um artigo na área de finanças quantitativas? Acesse aqui:
 https://www.outspokenmarket.com/diurnalis.html 

In [1]:
# Imports

import pandas as pd
import numpy as np
import yfinance as yf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from numpy_ext import rolling_apply as rolling_apply_ext

get_ipython().run_line_magic("matplotlib", "inline")
import warnings
warnings.filterwarnings("ignore")

# Data preparation

In [8]:
# Get the data

p1 = 252

ticker1 = "BRL=x" 
df1 = yf.download(ticker1, "2012-01-01", "2022-12-31")
df1["Returns"] = df1["Adj Close"].pct_change(1)
df1["Adj Low"] = df1["Low"] - (df1["Close"]-df1["Adj Close"])
df1["Adj High"] = df1["High"] - (df1["Close"]-df1["Adj Close"])
df1["Adj Open"] = df1["Open"] - (df1["Close"]-df1["Adj Close"])
df1["Target"] = df1["Returns"].shift(-1)
df1["MA"] = df1["Adj Close"].rolling(p1).mean()
df1 = df1.rename({"Adj Close": "AdjClose"}, axis = 1) 
df1.dropna(axis = 0, inplace = True) 
df1.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,AdjClose,Volume,Returns,Adj Low,Adj High,Adj Open,Target,MA
Date,,,,,,,,,,,,
2012-12-18,2.0945,2.1060,2.0784,2.0951,2.0951,0,0.004796,2.0784,2.1060,2.0945,-0.003389,1.948623
2012-12-19,2.0884,2.0910,2.0595,2.0880,2.0880,0,-0.003389,2.0595,2.0910,2.0884,-0.009004,1.949519
2012-12-20,2.0694,2.0750,2.0437,2.0692,2.0692,0,-0.009004,2.0437,2.0750,2.0694,0.000048,1.950309
2012-12-21,2.0693,2.0781,2.0531,2.0693,2.0693,0,0.000048,2.0531,2.0781,2.0693,-0.006282,1.951241
2012-12-24,2.0772,2.0888,2.0589,2.0563,2.0563,0,-0.006282,2.0589,2.0888,2.0772,0.009045,1.952185


In [9]:
# Simple funtion to get the polinomial slope of degree 1

def angle_c(x, y):
    return(np.polyfit(x, y, 1)[0])

In [10]:
window = 20
df1["Slope"] = rolling_apply_ext(angle_c, window, df1.MA.values, df1.AdjClose.values)

In [11]:
# Checking the slope

fig = make_subplots(rows = 2, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = df1.index, y = df1["AdjClose"]
                         , name = "Close", line = dict(color = "blue"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df1.index, y = df1["MA"]
                         , name = "MA 200"
                         , line = dict(color = "rgb(0,0,150)", dash = "dash", width = 1))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df1.index, y = df1["Slope"]
                         , name = "Slope", line = dict(color = "red"))
              , row = 2, col = 1)


fig.update_layout(height = 600, width = 800
                  , title_text = "Moving Slope - " + ticker1 + " - www.outspokenmarket.com"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis2_title = "Years"
                  , yaxis_title = "Close "
                  , yaxis2_title = "Slope"
                  , legend_title = "Indexes"
                  , font = dict(size = 15, color = "Black")
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df1.index[0]
                       , end = df1.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)

fig.show()

In [12]:
# Now, lets convert that slope to degrees

df1["Degrees"] = np.degrees(np.arctan(df1["Slope"]))

In [14]:
# And check it out in the chart

fig = make_subplots(rows = 3, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = df1.index, y = df1["AdjClose"]
                         , name = "Close", line = dict(color = "blue"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df1.index, y = df1["MA"]
                         , name = "MA"
                         , line = dict(color = "rgb(0,0,150)", dash = "dash", width = 1))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df1.index, y = df1["Degrees"]
                         , name = "Degrees", line = dict(color = "green"))
              , row = 2, col = 1)


fig.add_trace(go.Scatter(x = df1.index, y = df1["Slope"]
                         , name = "Slope", line = dict(color = "red"))
              , row = 3, col = 1)


fig.update_layout(height = 1000, width = 800
                  , title_text = "Moving Slope - " + ticker1 + " - www.outspokenmarket.com"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis3_title = "Years"
                  , yaxis_title = "Close"
                  , yaxis2_title = "Dregrees"
                  , yaxis3_title = "Slope"
                  , legend_title = "Indexes"
                  , font = dict(size = 15, color = "Black")
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df1.index[0]
                       , end = df1.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)

fig.show()

O que a literatura técnica diz sobre esse indicador?

- O Indicador de inclinação de regressão linear é um tipo de indicador de oscilador centralizado que é semelhante aos indicadores de momento;
- O momentum é positivo quando o slope (ou graus) está acima de 0 e negativo quando está abaixo de 0. Podemos usar este indicador para medir a força ou fraqueza e a direção do momentum.